# Setup

## Imports and Configuration

In [ ]:
%load_ext autoreload
%autoreload 2

import scanpy as sc
import os
import re
import json
from datetime import datetime
from anndata import AnnData
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import corescpy as cr

pd.options.display.max_columns = 100
pd.options.display.max_rows = 500
sc.settings.set_figure_params(dpi=100, frameon=False, figsize=(20, 20))

palette = "tab20"

# Column Names (from Metadata & To Be Created)
col_sample_id_o, col_sample_id = "Sample ID", "Sample"  # in metadata, new
col_subject = "Patient"  # in metadata file
# col_batch = "Slide"  # in metadata file
col_batch = None  # in metadata file
col_inflamed, col_stricture = "Inflamed", "Stricture"  # in metadata file
col_condition = "Condition"  # constructed from col_inflamed & col_stricture
col_fff = "file_path"  # column in metadata in which to store data file path
col_cell_type = "Annotation"  # for eventual cluster annotation column
col_tangram = "tangram_prediction"  # for future Tangram imputation annotation


def construct_file(sample=None, run="CHO-001", slide=None,
                   date=None, timestamp=None,
                   panel_id="TUQ97N", prefix="output-XETG00189",
                   project_owner="EA", directory=None):
    """Construct file path from information."""
    if "outputs" not in directory and os.path.exists(
            os.path.join(directory, "outputs")):
        directory = os.path.join(directory, "outputs")
    if sample is None:
        run = [run] if isinstance(run, str) else run
        panel_id = [panel_id] * len(run) if isinstance(
            panel_id, str) else panel_id
        fff = []
        for i, x in enumerate(run):
            d_x = os.path.join(directory, panel_id[i], x)
            fff += [os.path.join(d_x, y) for y in os.listdir(d_x)]
        return fff
    if isinstance(sample, str):
        sample = [sample]
    print(directory)
    panel_id, prefix, project_owner, slide, date, timestamp = [
        [x] * len(sample) if isinstance(x, str) else list(x) if x else x
        for x in [panel_id, prefix, project_owner, slide, date, timestamp]]
    run = [run] * len(sample) if isinstance(run, (str, int, float)) else run
    block = ["-".join(i) for i in zip(sample, panel_id, project_owner)]
    fff = [f"{prefix[i]}__{slide[i]}__{block[i]}" for i in range(len(sample))]
    if date is None or timestamp is None:
        for i, x in enumerate(fff):  # iterate current file stems
            ddd = os.path.join(directory, panel_id[i], run[i])
            print(ddd)
            matches = sum([x in d for d in os.listdir(ddd)])
            if  matches != 1:
                raise ValueError(f"{x} found in 0 or multiple file paths",
                                 f"\n\n{os.listdir(ddd)}")
            fff[i] = os.path.join(ddd, np.array(os.listdir(ddd))[np.where([
                x in d for d in os.listdir(ddd)])[0][0]])  # find match
    else:
        fff = [os.path.join(directory, panel_id[i], run[i],
                            f"{x}__{date[i]}__{timestamp[i]}")
               for i, x in enumerate(fff)]
    return fff


def perform_qc_concatenated(selves):
    hue = selves[0]._columns["col_sample_id"]
    ids = [str(s.rna.obs[hue].iloc[0]) for s in selves]
    patterns = [("MT-", "mt-"), ("RPS", "RPL", "rps", "rpl"), (
        "^HB[^(P)]", "^hb[^(p)]")]  # pattern matching for gene symbols
    patterns = dict(zip(["mt", "ribo", "hb"], patterns))  # dictionary
    names = dict(zip(["mt", "ribo", "hb"],
                        ["Mitochondrial", "Ribosomal", "Hemoglobin"]))
    p_names = [names[k] if k in names else k for k in patterns]  # "pretty"
    patterns_names = dict(zip(patterns, p_names))  # map abbreviated to pretty
    adata = AnnData.concatenate(
        *[x.rna for x in selves], join="outer", batch_key=hue,
        batch_categories=ids, index_unique=None, uns_merge="unique")
    qc_vars = []  # to hold mt, rb, hb, etc. if present in data
    for k in patterns:  # calculate MT, RB, HB counts
        gvars = adata.var_names.str.startswith(patterns[k])
        if any(gvars):
            qc_vars += [k]
        adata.var[k] = gvars
    pct_n = [f"pct_counts_{k}" for k in qc_vars]  # "% counts" variables
    cgs = selves[0]._columns["col_gene_symbols"] if selves[
        0].rna.var.index.values[0] not in selves[0].rna.var_names else None
    adata.obs = adata.obs.astype({col_condition: "category"})
    ggg = list(set(genes).intersection(adata.var_names))
    ctm = list(set(["n_genes_by_counts", "total_counts", "log1p_total_counts",
                    "cell_area", "nucleus_area"]
                ).intersection(adata.obs.columns))
    vam = pct_n + ctm + [hue]  # QC variable names
    mets_df = adata.obs[vam].rename_axis("Metric", axis=1).rename(
        {"total_counts": "Total Counts in Cell", "cell_area": "Cell Area",
        "nucleus_area": "Nucleus Area",
        "n_genes_by_counts": "Genes Detected in Cell",
        "log1p_total_counts": "Log-Normalized Total Counts",
        **patterns_names}, axis=1)  # rename
    fff = sb.pairplot(
        mets_df, hue=hue, height=3, diag_kind="hist",
        plot_kws=dict(marker=".", linewidth=0.05))  # pair
    return adata

## Options

In [ ]:
# Main Directories
# Replace manually or mirror my file/directory tree in your home (`ddu`)
ddu = os.path.expanduser("~")
ddm = "/mnt/cho_lab" if os.path.exists("/mnt/cho_lab") else "/mnt"  # Spark?
ddl = f"{ddm}/disk2/elizabeth/data/shared-xenium-library" if (
    "cho" in ddm) else os.path.join(ddu, shared-xenium-library)
ddx = f"{ddm}/bbdata0/xenium"  # mounted drive Xenium folder
out_dir = os.path.join(ddl, "outputs", "TUQ97N", "nebraska")  # None = no save
d_path = "/mnt/cho_lab/disk2/elizabeth/data"  # for other data
file_ann = os.path.join(ddu, "corescpy/examples/annotation_guide.xlsx")  # AG
file_mdf = os.path.join(ddl, "Xenium_Samples_03152024.xlsx")  # metadata

# Annotation & Tangram Imputation
col_assignment = "Bin"  # which column from annotation file to use
# col_cell_type_sc, file_sc = "ClusterAnnotation", str(
#     f"{d_path}/2023-05-12_CombinedCD-v2_ileal_new.h5ad")
col_cell_type_sc, file_sc = "cell_type", f"{d_path}/elmentaite_ileal.h5ad"

# Directories & Metadata
run = "CHO-001"
# run = ["CHO-001", "CHO-002", "CHO-005", "CHO-006"]
# samples = ["50007B2"]
# samples = "all"
samples = ["50452A"]
# samples = ["50452B"]
# samples = ["50452A", "50452B", "50452C"]
# samples = ["50564A4", "50618B5"]

# Input/Output Options
plot_all_qc = False
reload = True

# Computing Resources
gpu = False
sc.settings.n_jobs = 4
sc.settings.max_memory = 150

# Processing & Clustering Options
resolution = 0.5
min_dist = 0.3
n_comps = 50
method_norm = "log"
# custom_thresholds = {col_qscore: [, None]}
genes_subset, use_highly_variable = True, False  # genes to use in clustering
kws_pp = dict(cell_filter_pmt=None, cell_filter_ncounts=[50, None],
              cell_filter_ngene=[10, None], gene_filter_ncell=[3, None],
              gene_filter_ncounts=[3, None], custom_thresholds=None,
              kws_scale=dict(max_value=10, zero_center=True),
              method_norm=method_norm)  # preprocessing keyword arguments

# After this point, no more options to specify
# Just code to infer the data file path from your specifications
# and construct argument dictionaries and manipulate metadata and such.

# Read Metadata & Other Information
annot_df = pd.read_excel(file_ann)
metadata = pd.read_excel(file_mdf, dtype={"Slide ID": str}).rename(
    {"Name": col_subject, "Slide ID": col_batch,
     "Inflammation Status": col_inflamed}, axis=1)
if samples not in ["all", None]:  # subset by sample ID?
    metadata = metadata.set_index(col_sample_id_o).loc[samples].reset_index()

# Construct Clustering Argument Keyword Dictionaries
if genes_subset is True:
    genes_subset = list(annot_df.iloc[:, 0])
kws_umap = dict(min_dist=min_dist, method="rapids" if gpu else "umap")
kws_cluster = dict(use_gpu=gpu, kws_umap=kws_umap, kws_neighbors=None,
                   use_highly_variable=use_highly_variable, n_comps=n_comps,
                   genes_subset=genes_subset, resolution=resolution)

# Revise Metadata & Construct Variables from Options
metadata.loc[:, col_stricture] = metadata[col_stricture].replace(
    {"yes": "Stricture", "no": "None"})  # stricture column
metadata.loc[:, col_condition] = metadata.apply(lambda x: "Stricture" if x[
    col_stricture] == "Stricture" else x[col_inflamed].capitalize(), axis=1)
metadata.loc[:, col_sample_id] = metadata[[col_condition, col_sample_id_o]
                                          ].apply("-".join, axis=1)
metadata = metadata.set_index(col_sample_id)
fff = np.array(construct_file(run=run, directory=ddx))
samps = np.array([i.split("__")[2].split("-")[0] for i in fff])
for x in metadata[col_sample_id_o]:
    metadata.loc[metadata[col_sample_id_o] == x, col_fff] = fff[np.where(
        samps == x)[0][0]] if len(np.where(samps == x)[0]) > 0 else np.nan
metadata = metadata.dropna(subset=[col_fff])
file_path_dict = dict(zip(metadata.index.values, metadata["file_path"]))
kws_init = dict(col_batch=col_batch, col_subject=col_subject,
                col_sample_id=col_sample_id, col_cell_type=col_cell_type)
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Annotation File
assign = pd.read_excel(file_ann, index_col=0).dropna(
    subset=col_assignment).rename_axis("Gene")
marker_genes_dict = dict(assign.reset_index().groupby(col_assignment).apply(
    lambda x: list(pd.unique(x.Gene))))  # to marker dictionary

# Print Metadata
metadata

# Genes
# genes = ["CDKN1A", "CDKN2A", "TP53", "PLAUR", "PTGER4", "FTL", "IL6ST"]
# cell_types = ["ILC3", "LTi-like NCR+ ILC3", "LTi-like NCR- ILC3",
#               "ILCP", "Macrophages", "Stem cells"]
# palette = ["r", "tab:pink", "m", "b", "tab:brown", "cyan"]
# High in inf. vs. un
# OSM
# IL13
# IL1B
# IL6
# TNF
# S100A8
# S100A9
# ------------------------------
# High in stricture vs inf/un
# PDGFRA
# IL6ST
# PTPN1x
# IFNG

# Data

## Loading

In [ ]:
%%time

# Load Spatial Data
selves = [None] * metadata.shape[0]
for i, x in enumerate(metadata.index.values):
    selves[i] = cr.Spatial(metadata.loc[x][col_fff], library_id=x, **kws_init)
    for j in metadata:  # add metadata for subject
        selves[i].rna.obs.loc[:, j] = str(metadata.loc[x][j])  # add metadata

# Concatenate AnnData Objects & Perform QC
if plot_all_qc is True:
    adata = perform_qc_concatenated(selves)

# Reload Processed & Clustered Data (Optionally)
suff = str(f"res{re.sub('[.]', 'pt', str(resolution))}_dist"
           f"{re.sub('[.]', 'pt', str(min_dist))}_npc{n_comps}")  # file end
if reload is True:
    for i, s in enumerate(selves):
        f_o = os.path.join(out_dir, f"{x}__{suff}")
        s.rna.obs.loc[:, "out_file"] = f_o
        s.update_from_h5ad(file=f_o, file_path_markers=f_o + "_markers.csv")

## Explore Objects

In [ ]:
# Check Genes
for s in selves:
    _ = s.read_panel(directory=str(metadata.loc[s._library_id][col_fff]))

# General Information
[s.print() for s in selves]

# Processing

## Preprocessing

In [ ]:
%%time

# Print Pre-Processing Specifications
print(kws_pp)

# Perform Pre-Processing
if reload is False:
    for s in selves:
        _ = s.preprocess(**kws_pp, figsize=(15, 15))

# Check Remaining Genes
for s in selves:
    _ = s.read_panel(directory=str(metadata.loc[s._library_id][col_fff]))

## Clustering

In [ ]:
for i, s in enumerate(selves):
    _ = s.preprocess(**kws_pp, figsize=(15, 15))
    f_o = os.path.join(out_dir, s._library_id) if (
        out_dir) else None  # file/prefix for saved object & other outputs
    s.rna.obs.loc[:, "out_file"] = f_o
    for i in zip([0.5, 0.75, 1.5], [0.5, 0.3, 0], [30, 30, 30]):
        # Variable Name & Output File Construction
        kws = {**kws_cluster, "resolution": i[0], "kws_umap": {
            **kws_cluster["kws_umap"], "min_dist": i[1]}, "n_comps": i[2]}
        suff = str(f"res{re.sub('[.]', 'pt', str(kws['resolution']))}_dist"
             f"{re.sub('[.]', 'pt', str(kws['kws_umap']['min_dist']))}"
             f"_npc{kws['n_comps']}")  # file path suffix
        f_m = f"{f_o}_{x}_markers.csv" if out_dir else None
        cct, f_i = f"leiden_{suff}", f"{s._library_id}_{suff}"  # specific run
        _ = s.cluster(**kws, key_added=cct, out_file=f_o)
        s.write_clusters(out_dir, col_cell_type=cct, overwrite=True)  # XE
        _ = s.find_markers(col_cell_type=cct, kws_plot=False, out_file=f_m)
        _ = s.plot_spatial(color=cct)

In [ ]:
%%time

suff = str(f"res{re.sub('[.]', 'pt', str(kws_cluster['resolution']))}_dist"
           f"{re.sub('[.]', 'pt', str(kws_cluster['kws_umap']['min_dist']))}"
           f"_npc{n_comps}")  # file path suffix
if reload is False:
    for i, s in enumerate(selves):

            # Output File Path for Object
            f_o = os.path.join(out_dir, f"{s._library_id}__{suff}") if (
                out_dir is not None) else None  # output file
            s.rna.obs.loc[:, "out_file"] = f_o  # store output path in object

            # Cluster, Find Markers, & Plot
            _ = s.cluster(**kws_cluster, out_file=f_o)
            _ = s.find_markers(col_cell_type="leiden", kws_plot=False,
                               out_file=f_o + "_markers.csv" if f_o else None)
            _ = s.plot_spatial(color="leiden")

## Annotation 

In [ ]:
%%time

col = "Annotation_GEX"  # for method 2
# col = None  # skip method 2
if reload is False:
    for i, s in enumerate(selves):
        _ = s.annotate_clusters(file_ann, col_cell_type="leiden",
                                col_annotation=col_cell_type,
                                out_file=str(s.rna.obs.out_file.iloc[0]))
        if col:  # method 2
            _ = s.annotate_clusters(file_ann, col_annotation=col,
                                    renaming=True, method="GEX")
        # s._columns["col_cell_type"] = col + "_GEX"

# Manual Annotations
# annots = {
#     0: {"0": "Myofibroblast", "1": "Goblet", "2": "Cycling B",
#         "3": "Fibroblast", "4": "Endothelial",
#         # "5": "Goblet",
#         "5": "Epithelial",
#         # "6": "Goblet (Best, 2+?)", "7": "Myeloid (Macrophage? DC?)",
#         "6": "Epithelial", "7": "Myeloid",
#         "8": "Mesenchymal-Fibrolastic-Stromal 3",
#         "9": "Plasma", "10": "T Cell", "11": "Macrophage, LYVE1+",
#         "12": "Pericyte", "13": "Endothelial Lymphatic",
#         "14": "Mast", "15": "Mesenchymal-Myocytic-Myofibroblast"},
#     1: {"0": "Contractile Pericyte",
#         "1": "Mesenchymal-Fibroblastic-Stromal 1,3",
#         "2": "Endothelial", "3": "B Cell", "4": "T Cell",
#         "5": "Mesenchymal-Fibroblastic-Stromal 1", "6": "Myofibroblast",
#         "7": "Epithelial", "8": "Plasma", "9": "Macrophage",
#         "10": "Myeloid", "11": "Mast",
#         "12": "Lymphatic Endothelial", "13": "Neural"},
#     2: {"0": "B Cell", "1": "Mesenchymal", "2": "T/NK Cell",
#         "3": "Mesenchymal-Fibroblastic-Transitional Stromal 1",
#         "4": "Smooth Muscle", "5": "Endothelial", "6": "Macrophage",
#         "7": "Lymphatic Endothelial",
#         "8": "Smooth Muscle/Pericyte/Myofibroblast?",
#         "9": "IgA Plasma", "10": "Cycling B Cell",
#         "11": "Epithelial-Enterocyte-Tuft?", "12": "Neural"}
# }

# ONLY FOR RESOLUTION = 0.75,
# JUST CELL TYPE GENES USED IN CLUSTERING,
# MIN_DIST = 0.80
# rename_ann = {
#     str("Activated CD4 T_Branch A4 (IN)_CLP_Colonocyte_Contractile Pericyte"
#         " (PLN+)_D Cells (SST+)_FCRL4+ Memory B_FDC_Fetal Arterial EC_Glia"
#         " 2 (ELN+)_ILCP_Lymphoid DC_Macrophages_Mast Cell_Microfold Cell_N"
#         " Cells (NTS+)_NK Cell_Pericyte_SELL+ CD8 T_Stromal 1 (ADAMDEC1+)"
#         "_TRGV2 Gdt_cDC2"): "Mixed Immune",
#     str("Activated CD4 T_BEST2+ Goblet Cell_Branch A4 (IN)_CLP_Colonocyte_"
#         "Contractile Pericyte (PLN+)_D Cells (SST+)_FCRL4+ Memory B_FDC"
#         "_Fetal Arterial EC_Glia 2 (ELN+)_ILCP_Immature B_Macrophages"
#         "_Microfold Cell_NK Cell_Pericyte_SELL+ CD8 T_Stromal 1 (ADAMDEC1+)"
#         "_TRGV2 Gdt_cDC2_mLTo"): "Immune (Lymphoid?)",
#     str("Activated CD4 T_Branch A4 (IN)_CLP_Colonocyte_Contractile Pericyte"
#         " (PLN+)_D Cells (SST+)_FCRL4+ Memory B_FDC_Fetal Arterial EC_Glia"
#         " 2 (ELN+)_ILCP_Lymphoid DC_Macrophages_Mast Cell_Microfold Cell_N"
#         " Cells (NTS+)_NK Cell_Pericyte_SELL+ CD8 T_Stromal 1 (ADAMDEC1+)_"
#         "TRGV2 Gdt_cDC2"): "Mixed Immune",
#     str("Activated CD4 T_Branch A4 (IN)_CD8 Tmem_CLP_FCRL4+ Memory B_FDC_"
#         "Fetal Arterial EC_Glia 2 (ELN+)_ILCP_Lti-like NCR+ ILC3_Lymphoid"
#         " DC_Macrophages_Mast Cell_Megakaryocyte_Microfold Cell_NK Cell_"
#         "Pericyte_SELL+ CD8 T_TRGV2 Gdt_mLTo"): "Mixed Immune",
#     str("Activated CD4 T_Branch A4 (IN)_CLP_Distal Progenitor_FCRL4+ Memory"
#         " B_FDC_Fetal Arterial EC_Glia 2 (ELN+)_ILC3_Lti-like NCR+ ILC3_"
#         "Macrophages_Mast Cell_Microfold Cell_Monocytes_Pericyte_Proximal "
#         "Progenitor_SELL+ CD8 T_Stromal 1 (ADAMDEC1+)_"
#         "TRGV2 Gdt"): "B_T_ILC",
#     str("Angiogenic Pericyte_Immature Pericyte_LYVE1+ Macrophage"
#         ): "Pericyte_Macrophage"}

rename_ann = None

# Rename Annotations
if rename_ann is not None:
    for i, s in enumerate(selves):
        s.rna.obs.loc[:, f"{col}_Auto"] = s.rna.obs[col].copy()
        s.rna.obs.loc[:, col] = s.rna.obs[col].replace(
            rename_ann if isinstance(rename_ann, dict) else rename_ann[i])
    [print(list(s.rna.obs[col].unique())) for s in selves]

## Plot Spatial

In [ ]:
for s in selves:
    print(f"\n\n{'=' * 80}\n{s._library_id}\n{'=' * 80}\n\n")
    for c in [j for j in ["leiden", col_cell_type, col] if j in s.rna.obs]:
        s.plot_spatial(title=s._library_id, color=c)

## Annotation by Imputation (Tangram)

In [ ]:
adata_sc = sc.read(file_sc)  # read whole transcriptome data for imputation
for i, s in enumerate(selves):
    _ = s.impute(
        adata_sc.copy(), col_cell_type=col_cell_type_sc,
        # mode="cells",
        mode="clusters",
        markers=None,  # use all overlap
        # markers=200,
        plot=False, plot_density=False, plot_genes=None,
        col_annotation=col_tangram,
        out_file=str(s.rna.obs.out_file.iloc[0]))
        # adata_sp_new, sdata, adata_sc_n, ad_map, df_compare, fig = out

## Write Xenium Explorer Files

In [ ]:
s.write(str(s.rna.obs.out_file.iloc[0]))

In [ ]:
col_leid = [j for j in s.rna.obs if "leiden" in j]
for i, s in enumerate(selves):
    for c in [j for j in col_leid + [col_cell_type, col] if j in s.rna.obs]:
            s.write_clusters(out_dir, col_cell_type=c, overwrite=True,
                             file_prefix=str(s.rna.obs.out_file.iloc[0]))

## Marker Plots (Additional)

In [ ]:
_ = self.plot(kind="heat", swap_axes=True, vmax=10,
              marker_genes_dict=marker_genes_dict, layer="counts")

## Concatenate AnnData

In [ ]:
hue = selves[0]._columns["col_sample_id"]
ids = [str(s.rna.obs[hue].iloc[0]) for s in selves]
adata = AnnData.concatenate(
    *[x.rna for x in selves], join="outer", batch_key=hue,
    batch_categories=ids, index_unique=None, uns_merge="unique")

## Preliminary Exploration

In [ ]:
# for s in selves:
#     _ = s.read_panel(directory=metadata[s._library_id]["file_path"])
# for s in selves:
#     s.plot_umap(color=genes + [self._columns["col_cell_type"]],
#                 layer="scaled", cmap="coolwarm", vcenter=0)
cct = selves[0]._columns["col_cell_type"]
for g in [["CDKN1A", "CDKN2A", "TP53", "PLAUR", "PTGER4", "FTL", "IL6ST"],
          ["CSF2RB", "CSF2RA", "IL7R", "TBX21", "KLRB1", "CCL5", "HSP90AA1",
           "RORC", "IL23R", "KIT", "CCL20", "TCF7", "LTB", "IL22", "IFNG"],
          ["PDGFRA", "IL6ST", "PTPN1", "IFNG"]]:
    ggg = list(set(g).intersection(adata.var_names))
    cr.pl.plot_cat_split(adata, col_inflamed, col_cell_type=cct,
                         genes=ggg, kind="box")
    cr.pl.plot_cat_split(adata, col_inflamed, col_cell_type=cct,
                         genes=ggg, columns=col_stricture, kind="box")

# Analyses

## Centrality Scores

In [ ]:
%%time

for s in selves:
    s.calculate_centrality(n_jobs=sc.settings.n_jobs)

## Neighborhood Enrichment Analysis

In [ ]:
%%time

for s in selves:
    _ = s.calculate_neighborhood(figsize=(60, 30))

## Cell Type Co-Occurrence

In [ ]:
%%time

for s in selves:
    _ = s.find_cooccurrence(figsize=(60, 20), kws_plot=dict(wspace=3))

## Spatial Clustering

In [ ]:
for s in selves:
    _ = s.cluster_spatial(key_added="leiden_spatial")

## Spatially-Variable Genes

In [ ]:
%%time

kws = dict(kws_plot=dict(legend_fontsize="large"), figsize=(15, 15))
for s in selves:
    _ = s.find_svgs(genes=genes, method="moran", n_perms=10, **kws)

## Receptor-Ligand Interactions

In [ ]:
%%time

for s in selves:
    kss, ktt = None, None
    _ = s.calculate_receptor_ligand(
        col_condition=False, p_threshold=0.01, remove_ns=True,
        figsize=(30, 10), top_n=25, key_sources=kss, key_targets=ktt)
    # s.calculate_receptor_ligand_spatial()

## Cell Type Composition 

In [ ]:
for s in selves:
    s.run_composition_analysis()

## GEX

In [ ]:
for s in selves:
    s.plot_spatial(color=["TNF", "IL23", col_cell_type])

In [ ]:
_ = self.calculate_spatial_distance("LTi-like NCR+ ILC3", genes="CSF2RB")

In [ ]:
adata.X = adata.layers["counts"]
sq.tl.var_by_distance(
    adata, "LTi-like NCR+ ILC3", col_cell_type, library_key=col_sample_id,
    design_matrix_key="design_matrix", covariates=["Patient", "Inflamed"],
    metric="euclidean", spatial_key=self._spatial_key, copy=False)
sq.pl.var_by_distance(
    adata=adata, var=["CSF2RB", "CSF2RA", "IL7R"],
    anchor_key="LTi-like NCR+ ILC3",
    covariate="Inflamed")

## Compare GEX

In [ ]:
# goi = ["CSF2", "CSF2RB", "CSF2RA", "CDKN1A", "CDKN2A", "TP53", "PLAUR"]
# goi = ["CDKN1A", "CDKN2A", "TP53", "PLAUR", "PTGER4", "FTL", "IL6ST", "CSF2",
#        "CSF2RB", "CSF2RA", "IL7R", "TBX21", "KLRB1", "CCL5", "HSP90AA1",
#        "RORC", "IL23R", "KIT", "CCL20", "TCF7", "LTB", "IL22", "IFNG"]
# goi = genes

# fig, axs = selves[0].plot_compare_spatial(selves[1:], goi)

# fig, axs = plt.subplots(len(goi) + 1, len(selves),
#                         figsize=(5 * len(goi), 20 * len(selves)))
# for j, s in enumerate(selves):
#     for i, g in enumerate(goi + [s._columns["col_cell_type"]]):
#         s.plot_spatial(ax=axs[i, j], color=None if i == 0 else goi[i - 1],
#                        cmap="magma", layer=s._layers["log1p"])
#         if i == 0:
#             axs[i, j].set_title(s._library_id)
# plt.subplots_adjust(wspace=0.3)
# fig.show()


### Senescence

In [ ]:
goi = ["CDKN1A", "CDKN2A", "TP53", "PLAUR", "PTGER4", "FTL", "IL6ST"]
fig, axs = selves[0].plot_compare_spatial(selves[1:], goi, wspace=0.3)

### CSF2RB

In [ ]:
%%time

goi = ["CSF2RB", "CSF2RA", "IL7R", "TBX21", "KLRB1", "CCL5", "HSP90AA1",
       "RORC", "IL23R", "KIT", "CCL20", "TCF7", "LTB", "IL22", "IFNG"]
fig, axs = selves[0].plot_compare_spatial(selves[1:], goi, figsize=(40, 70),
                                          wspace=0.1)

### High in Inflamed vs. Un-Inflamed? 

In [ ]:
%%time

goi = ["OSM", "IL13", "IL1B", "IL6", "TNF", "S100A8", "S100A9"]
fig, axs = selves[0].plot_compare_spatial(selves[1:], goi
                                          wspace=0.5, figsize=(40, 50)))

### High in Stricture vs. Uni-Inflamed?

In [ ]:
%%time

goi = ["PDGFRA", "IL6ST", "PTPN1", "IFNG"]
fig, axs = selves[0].plot_compare_spatial(selves[1:], goi,
                                          wspace=0.5, figsize=(40, 50))

# Workspace

## Mast-Neural

In [ ]:
list(selves[0].rna.obs[col_cell_type].unique())

In [ ]:
key_mast_neural = ["Mast cell", "Glia 1 (DHH+)", "SMC"]

In [ ]:
for s in selves:
    _ = s.find_cooccurrence(figsize=(30, 20), kws_plot=dict(wspace=3),
                            key_cell_type=key_mast_neural)

In [ ]:
def _get_data(adata: AnnData, cluster_key: str, func_name: str,
              attr: str = "uns", **kwargs):

    key = getattr(Key.uns, func_name)(cluster_key, **kwargs)

    try:
        if attr == "uns":
            return adata.uns[key]
        elif attr == "obsm":
            return adata.obsm[key]
        else:
            raise ValueError(f"attr must be either 'uns' or 'obsm', got {attr}")
    except KeyError:
        raise KeyError(
            f"Unable to get the data from `adata.uns[{key!r}]`. "
            f"Please run `squidpy.gr.{func_name}(..., cluster_key={cluster_key!r})` first."
        ) from None

In [ ]:
from __future__ import annotations

from pathlib import Path
from types import MappingProxyType
from typing import (
    TYPE_CHECKING,
    Any,
    Literal,
    Mapping,
    Sequence,
    Union,  # noqa: F401
)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from anndata import AnnData
from matplotlib.axes import Axes

from squidpy._constants._constants import RipleyStat
from squidpy._constants._pkg_constants import Key
from squidpy._docs import d
from squidpy.gr._utils import (
    _assert_categorical_obs,
    _assert_non_empty_sequence,
    _get_valid_values,
)
from squidpy.pl._color_utils import Palette_t, _get_palette, _maybe_set_colors
from squidpy.pl._utils import _heatmap, save_fig

In [ ]:
clusters = key_mast_neural
cluster_key = col_cell_type
adata = selves[0].rna.copy()
figsize = (20, 20)
legend_kwargs = {}
dpi = 100

In [ ]:
    occurrence_data = _get_data(adata, cluster_key=cluster_key,
                                func_name="co_occurrence")

    legend_kwargs = dict(legend_kwargs)
    if "loc" not in legend_kwargs:
        legend_kwargs["loc"] = "center left"
        legend_kwargs.setdefault("bbox_to_anchor", (1, 0.5))

    out = occurrence_data["occ"]
    interval = occurrence_data["interval"][1:]
    categories = adata.obs[cluster_key].cat.categories

    clusters = categories if clusters is None else clusters
    clusters = _assert_non_empty_sequence(clusters, name="clusters")
    clusters = sorted(_get_valid_values(clusters, categories))

    palette = _get_palette(adata, cluster_key=cluster_key,
                           categories=categories, palette=None)

    fig, axs = plt.subplots(
        1,
        len(clusters),
        figsize=(5 * len(clusters), 5) if figsize is None else figsize,
        dpi=dpi,
        constrained_layout=True,
    )
    axs = np.ravel(axs)  # make into iterable

    for g, ax in zip(clusters, axs):
        idx = np.where(categories == g)[0][0]
        df = pd.DataFrame(out[idx, :, :].T, columns=categories).melt(var_name=cluster_key, value_name="probability")
        df["distance"] = np.tile(interval, len(categories))

        sns.lineplot(
            x="distance",
            y="probability",
            data=df,
            dashes=False,
            hue=cluster_key,
            hue_order=categories,
            palette=None,
            ax=ax
        )
        ax.legend(**legend_kwargs)
        ax.set_title(rf"$\frac{{p(exp|{g})}}{{p(exp)}}$")
        ax.set_ylabel("value")

In [ ]:
import squidpy as sq

sq.pl.co_occurrence(
    s.rna,
    cluster_key=col_cell_type,
    clusters=key_mast_neural[0],
    figsize=(10, 10),
)

sq.pl.co_occurrence(
    s.rna,
    cluster_key=col_cell_type,
    clusters=key_mast_neural[1],
    figsize=(10, 10),
)

In [ ]:
for s in selves:
    kss, ktt = key_mast_neural, key_mast_neural
    _ = s.calculate_receptor_ligand(
        col_condition=False, p_threshold=0.01, remove_ns=True,
        figsize=(30, 10), top_n=25, key_sources=kss, key_targets=ktt)

## Image Adding/Displaying

In [ ]:
paths_he = []
for x in metadata.index.values:
    paths_he += [os.path.join(metadata.loc[x][
        col_fff], "aux_outputs/image_he.tif")]  # H&E paths




import tifffile as tiff

p = paths_he[i]

img = tiff.imread(p)
mg = PIL.moveaxis(img, -1, 1)[0]
img = Image2DModel.parse(img, transformations={"global": Identity()},
                         dims=("c", "y", "x"))
self.adata.add_image("morphology_he", img)

In [ ]:
from PIL import Image, ImageFile
import spatialdata

Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True

img = Image.open(paths_he[i]).convert("RGB")
# img = np.asarray(Image.open(paths_he[i]).convert(
#     "RGB"), dtype=np.float32) / 255
self.adata.add_image("morphology_he",
                     spatialdata.models.Image2DModel.parse(img))

In [ ]:
self = selves[1]

plotter = spatialdata_plot.pl.basic.PlotAccessor(self.adata)
dir(plotter)

In [ ]:
self.adata.points
self.adata.shapes["cell_boundaries"]
self.adata.points["transcripts"].feature_name.compute()

# transcripts = self.read_parquet()
# transcripts

sdata_im = self.adata.aggregate(values="transcripts",
                                by="cell_boundaries", agg_func="count")
sdata_im

In [ ]:
cr.pl.plot_tiff()

self.adata.table.uns[self._spatial_key]["Stricture-50452C"]["images"][
    "Stricture-50452C___morphology_mip_scale3"].show()

self.adata.pl.render_shapes().pl.show()

self.adata.pl.render_images().pl.render_shapes().pl.show()

fig, axs = plt.subplots(ncols=3, figsize=(12, 3))
sdata.render_images().pl.show(ax=axs[0])
sdata.pl.render_labels().pl.show(ax=axs[1])
sdata.pl.render_shapes().pl.show(ax=axs[2])
sdata.pl.render_images().pl.render_labels().pl.render_shapes().pl.show()

In [ ]:
from spatialdata_io.readers.xenium import xenium_aligned_image

img = xenium_aligned_image("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/outputs/TUQ97N/CHO-001/output-XETG00189__0010700__50452A-TUQ97N-EA__20240126__205019/aux_outputs/50452-A (FFPE) - 2024-01-16 11.36.14.tif", None)
dir(img)

In [ ]:
import spatialdata_plot

plotter = spatialdata_plot.pl.basic.PlotAccessor(self.adata)
# # plotter.render_images("morphology_focus").pl.show()
# # plotter.render_points(elements="transcripts", color="CDKN1A").pl.show()
# plotter.render_shapes(elements="cell_boundaries", color="green").pl.show()
plotter.render_shapes(elements="cell_boundaries",
                      color="Annotation").pl.show("global")

plotter.render_shapes(elements="cell_boundaries").pl.show()
plotter.render_points(elements="transcripts", color="feature_name",
                      groups="CDKN1A").pl.show("global")

plotter.render_points(elements=None, color="green",
                      groups=["STAT1"]).pl.show("global")

In [ ]:
self.show(kinds=["images", "shapes"], elements=[
    "cell_boundaries", "morphology_mip"],
          color={"morphology_mip": "Annotation"})
# self.show(kinds="all", elements=["IL23", "cell_boundaries", "morphology_mip"])

## Bare Squidpy

In [ ]:
# import squidpy as sq
# import scanpy as sc
# import spatialdata_io as sdio

# col_cell_type = "leiden"
# n_jobs = 4
# file = file_path_dict[list(file_path_dict.keys())[0]]

# sdata = sdio.xenium(file, n_jobs=n_jobs)

# sq.gr.spatial_neighbors(sdata.table)
# sc.pp.pca(sdata.table)
# sc.pp.neighbors(sdata.table)
# sc.tl.leiden(sdata.table)
# sq.gr.nhood_enrichment(sdata, cluster_key=col_cell_type, seed=1618)
# sq.gr.co_occurrence(sdata, cluster_key=col_cell_type, n_jobs=n_jobs)
# sq.pl.spatial_scatter(sdata, color=col_cell_type, shape="hex")
# cr.pl.plot_cooccurrence(sdata, col_cell_type=col_cell_type)
# sq.gr.spatial_autocorr(sdata, mode="moran", n_jobs=n_jobs)
# sq.pl.spatial_scatter(adata, color=genes + [col_cell_type], shape="hex")
# sdata[0].images["morphology_mip"]

## Whole Process Example

In [ ]:
# self = selves[0]
# _ = self.preprocess(**kws_pp)
# _ = self.cluster(**kws_cluster)
# # self.adata.write(out_files[i], overwrite=True)  # optional, write object
# self.calculate_centrality(n_jobs=4)
# self.find_cooccurrence(figsize=(60, 20), kws_plot=dict(wspace=3))
# self.find_svgs(genes=genes, method="moran", n_perms=10, kws_plot=dict(
#     legend_fontsize="large"), figsize=(15, 15))
# self.calculate_receptor_ligand(col_condition=False, p_threshold=0.001,
#                                remove_ns=True, figsize=(20, 20))

## Re-Name Files in Standard Form Using Object

In [ ]:
# fff = list(pd.Series([x if "h5ad" in x else np.nan for x in os.listdir(
#     out_dir)]).dropna())
# frn = []
# for x in fff:
#     ann = sc.read(os.path.join(out_dir, x))
#     kwu = re.sub("{", "", re.sub("}", "", str(ann.obs.iloc[0].kws_umap))
#                  ).split(", ")
#     kwu = np.array(kwu)[np.where(["min_dist" in x for x in kwu])[
#         0]][0].split(": ")[1]
#     frn += [str(f"{str(ann.obs.iloc[0][col_sample_id])}__"
#                 f"res{str(ann.obs.iloc[0].resolution)}"
#                 f"_dist{kwu}_npc{ann.varm['PCs'].shape[1]}")]
# frd = dict(zip([os.path.join(out_dir, i) for i in fff], [os.path.join(
#     out_dir, re.sub("[.]", "pt", re.sub("[.]h5ad", "", i)) + ".h5ad")
#                                                          for i in frn]))
# frd
# # for x in frd:
# #     os.system(f"mv {x} {frd[x]}")